In [1]:
#Only run in colab
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/Github/flywire

ModuleNotFoundError: No module named 'google.colab'

In [2]:
import numpy as np
import scipy as sp
from utility import M,F, benchmark_mapping, get_best_connected, invert, score, get_highest_out_degrees
rng = np.random.default_rng(0)

In [3]:
m,n = 50,500
H_M,m_labels = get_best_connected(M,n)
H_F,f_labels = get_best_connected(F,n)
H_M = H_M.todense()
H_F = H_F.todense()
H_F[np.arange(n),np.arange(n)] = 0
H_M[np.arange(n),np.arange(n)] = 0
np.count_nonzero(H_M)/n**2, np.count_nonzero(H_F)/n**2

(0.224672, 0.123912)

In [63]:
result_matrix = np.zeros((n,n),dtype=int)
score_track = []
mapping_save = []

for k in range(500):

    mapping_M = rng.choice(n,m,replace=False)
    mapping_F = rng.choice(n,m,replace=False)

    for i in range(50):

        outer_unchanged = True

        if i % 2 == 0:
            G = H_F[mapping_F[:,None],mapping_F[None,:]]
            H = H_M
            mapping = mapping_M.copy()
        else:
            G = H_M[mapping_M[:,None],mapping_M[None,:]]
            H = H_F
            mapping = mapping_F.copy()

        unmapped = np.ones(n,dtype=bool)
        unmapped[mapping] = False
        top_score = score(G,H,mapping,require_surjective=False)
        
        for j in range(50):

            unchanged = True

            for u in rng.permutation(m):

                change_scores = find_best_move(G,H,mapping,u)
                w = (change_scores*unmapped).argmax()

                if change_scores[w] > change_scores[mapping[u]]:
                    top_score = top_score + change_scores[w]-change_scores[mapping[u]]

                    unmapped[mapping[u]] = True
                    unmapped[w] = False
                    mapping[u] = w
                    unchanged = False
                    outer_unchanged = False

            if unchanged:
                break
        
        if i % 2 == 0:
            mapping_M = mapping.copy()
        else:
            mapping_F = mapping.copy()

        if outer_unchanged:
            print(f"{k=}, {i=}, {top_score=}"+" "*20,end="\r")
            break

    A = H_F[mapping_F[:,None],mapping_F[None,:]]
    B = H_M[mapping_M[:,None],mapping_M[None,:]]

    score_track.append((score(G,H,mapping,require_surjective=False), score(A,A,np.arange(m)), score(B,B,np.arange(m))))
    mapping_save.append((mapping_F,mapping_M))
    result_matrix[mapping_F,mapping_M] += 1

KeyboardInterrupt: 

In [ ]:
np.concatenate([m_labels[mapping_M,None],f_labels[mapping_F,None]],axis=1)

In [12]:
A=M.todense()
B=F.todense()
N=A.shape[0]
A[np.arange(N),np.arange(N)]=0
B[np.arange(N),np.arange(N)]=0

In [76]:
G= B
H=A
mapping = np.load("mappings/Mapping-14-12-24 (5.82 Mio).npy")
score(G,H,mapping)

5821937

In [ ]:
from algorithms import get_scores_by_vertex, make_best_swap, find_best_move

scores_by_vertex = get_scores_by_vertex(G,H,mapping)
score_track=score(G,H,mapping)

for u in range(N):#range(1319,N):
    improvement = make_best_swap(G,H,mapping,u,scores_by_vertex)
    score_track += improvement
    print(f"{u=}, {score_track=}",end="\r")

scores_by_vertex.sum()/2, score(G,H,mapping)

KeyboardInterrupt: 

In [74]:
score(G,H,mapping)

5724030

In [70]:
main_mapping = mapping.copy()
np.save("mappings/Mapping-14-12-24 (5.82 Mio).npy",main_mapping)

In [5]:
rng = np.random.default_rng()
from algorithms import greedy_mapping, greedy_max
#start_mapping=-np.ones(N,dtype=int)
inverse_benchmark = invert(benchmark_mapping)
#start_mapping[f_labels[mapping_F]]=m_labels[mapping_M]
#random_set = rng.choice(N,500,replace=False)
#start_mapping[:500] = inverse_benchmark[:500]
mapping = greedy_max(H_F,H_M,start_mapping=None, c=0.1)

In [34]:
results = np.zeros((n,n),dtype=int)
for i in range(n):
    for j in range(n):
        if i==j:
            continue
        start_mapping = -np.ones(n,dtype=int)
        start_mapping[0]=i
        start_mapping[1]=j
        mapping=greedy_max(H_F,H_M,start_mapping,verbose=False,c=0)
        results[i,j] = score(H_F,H_M,mapping)
        print(f"{i=}",end="\r")

In [19]:
inverse_benchmark=main_mapping.copy()
m_labels=inverse_benchmark[f_labels]
H_M = (M[m_labels[:,None],m_labels[None,:]]).todense()
score(H_F,H_M,np.arange(n))

201352

In [ ]:
from utility import random_graph,shuffle_graph
from algorithms import greedy_max

G=B
H=A

mapping = main_mapping.copy()
high_score = score(G,H,mapping,require_surjective=False)
last_change = 0
save_count = 800

for i in range(5):
    start_mapping = -np.ones(n,dtype=int)
    #choice = np.concatenate([np.arange(100),rng.choice(np.arange(100,n),size=save_count,replace=False)])
    choice = rng.choice(n,size=save_count,replace=False)
    start_mapping[choice]=mapping[choice]

    next_mapping=greedy_max(G,H,start_mapping,verbose=False,c=0.1,injective=True)
    
    scores_by_vertex = get_scores_by_vertex(G,H,next_mapping)

    for _ in range(5):
        for u in rng.permutation(n):
            make_best_swap(G,H,mapping,u,scores_by_vertex)

    current_score = score(G,H,next_mapping,require_surjective=False)

    if current_score>high_score:
        high_score=current_score
        mapping = next_mapping
        last_change = i
    elif i-last_change >10:
        save_count += 1
        last_change = i

    print(f"{i=}, {high_score=}, {save_count=}",end="\r")

array([ 71,  95,  24, 107, 151,  49,  66,  21,  19, 177,  50, 198,   5,
         8,   7,  31,  70, 116,  29, 148, 145,  23, 105, 140, 111,  42,
        45,  35, 134, 114, 110, 196,   3,   9, 179, 135, 136,  55, 124,
        84,  92,  47, 169, 115,  51,  59, 117, 100, 121,  39,  62,  57,
       133,  54, 103, 142,  46,  82, 143, 158,   0, 199, 193, 183,  18,
       118,  98,  78, 131,  26, 168, 166,  41, 197,  25, 190, 176, 180,
       132,  20,   2, 195, 167,  86,  69,  27,  73, 187, 189, 139,  64,
       113, 112, 144,  80, 141,  15,  33,  87, 120,  76,  28,  53, 123,
       165, 102, 162, 185,  61, 104, 109, 175, 150, 125,  93,  99,  96,
       126, 157, 156, 163, 119, 181, 159,  60,  75,  65, 127,  10, 108,
       137,  22,  68, 178, 129, 160, 182, 122,  97,  85, 161,  36,  16,
        12, 101, 146,  91,  40,  81, 173,  67, 154, 147,  89,   4, 164,
         1,  83,  30, 153,  38, 184, 188,  56,  72, 171,  37,  11, 106,
       130,  17,  63,  79,  58,  77,  94,  44, 149, 152, 174,  7

In [ ]:
m=50
mapping_M = rng.choice(n,m,replace=False)
mapping_F = rng.choice(n,m,replace=False)

for i in range(50):

    outer_unchanged = True

    if i % 2 == 0:
        G = H_F[mapping_F[:,None],mapping_F[None,:]]
        H = H_M
        mapping = mapping_M.copy()
    else:
        G = H_M[mapping_M[:,None],mapping_M[None,:]]
        H = H_F
        mapping = mapping_F.copy()

    unmapped = np.ones(n,dtype=bool)
    unmapped[mapping] = False
    top_score = score(G,H,mapping,require_surjective=False)
    
    for j in range(50):

        unchanged = True

        for u in rng.permutation(m):

            change_scores = find_best_move(G,H,mapping,u)
            w = (change_scores*unmapped).argmax()

            if change_scores[w] > change_scores[mapping[u]]:
                top_score = top_score + change_scores[w]-change_scores[mapping[u]]

                unmapped[mapping[u]] = True
                unmapped[w] = False
                mapping[u] = w
                unchanged = False
                outer_unchanged = False

        if unchanged:
            break
    
    if i % 2 == 0:
        mapping_M = mapping.copy()
    else:
        mapping_F = mapping.copy()

    if outer_unchanged:
        print(f"{i=}, {top_score=}"+" "*20,end="\r")
        break

In [11]:
from utility import random_graph,shuffle_graph
from algorithms import get_scores_by_vertex, make_best_swap
n=500
G=random_graph(n,max=1,rng=rng)
H,shuffle=shuffle_graph(G,rng=rng)
mapping = rng.permutation(n)

scores_by_vertex = get_scores_by_vertex(G,H,mapping)
score_track=score(G,H,mapping)

print(score(G,G,np.arange(n)))

for j in range(50):
    for u in np.arange(n):
        improvement = make_best_swap(G,H,mapping,u,scores_by_vertex)
        score_track += improvement
    print(f"{j=}, {score_track=}"+" "*10,end="\r")

score(G,H,mapping),score(G,G,np.arange(n))

6237700


KeyboardInterrupt: 

In [71]:
G=B
H=A
start_mapping = -np.ones(N,dtype=int)
choice = rng.choice(N,size=8000,replace=False)
start_mapping[choice]=main_mapping[choice]

next_mapping=greedy_max(G,H,start_mapping,verbose=True,c=0,injective=True)

In [72]:
mapping=next_mapping
score(G,H,next_mapping)

5300886